#### Global Mountain Explorer (GME)
website:   https://rmgsc.cr.usgs.gov/gme/

data link: https://rmgsc.cr.usgs.gov/outgoing/ecosystems/Global/

Our purpose is manipulate this dataset according our model's demand. Dataset default nodata value is 0 and we want to change this. Over the land zero will represent flat surface and over other place(sea,ocean) nodata value will -32768.

vector to raster process with gdal

gdal_rasterize -l countries_border -burn 1.0 -tr 0.002083333333 0.002083333333 -a_nodata 0.0 -te lower_left upper right -ot Byte -of GTiff world_shp/countries_border.shp "world_shp/rasterize_shp.tif"

In [ ]:
# rasterize world shp file
gdal_rasterize -l countries_border -burn 1.0 -tr 0.01 0.01 -a_nodata 0.0 -te lower_left upper right -ot Byte -of GTiff world_shp/countries_border.shp "world_shp/rasterize_1km/rasterize_shp.tif"

In [ ]:
from osgeo import ogr, gdal, osr
import os 
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# we create tile from target image
img='../data/world_shp/rasterize_shp.tif'
output_path='../data/world_shp/tilesV2/world_shp_tile_'
input_path=img
ds = gdal.Open(img)
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize
# define your tile size
# it could be 256,512 or 1024. it depends on your scope
tile_size_x = 1024
tile_size_y = 1024
for i in range(0, xsize, tile_size_x):
    if xsize-i < tile_size_x:
        i=xsize-tile_size_x            
    for j in range(0, ysize, tile_size_y):
        if ysize-j < tile_size_y:
            j=ysize-tile_size_y

        com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(input_path)+ " " + str(output_path)  + str(i) + "_" + str(j) + ".tif"
        #print(com_string)
        os.system(com_string)

In [ ]:
img_tiles=glob.glob('../data/GlobalMountainsK3Binary/tilesV2/*.tif')
img_tiles=sorted(img_tiles)

In [ ]:
img_tiles[0:10]

In [ ]:
len(img_tiles)

In [ ]:
shp_tiles=glob.glob('../data/world_shp/tilesV2/*.tif')
shp_tiles=sorted(shp_tiles)

In [ ]:
len(shp_tiles)

In [ ]:
shp_tiles[0][-8:]

In [ ]:
shp_tiles[12][-11:]

In [ ]:
shp_tiles[12][-8:]==img_tiles[12][-8:]

In [ ]:
# delete nodata tiles in target folders.
# we define no-land area from raster that produced from shp file
count=0
for index, img in enumerate(shp_tiles,0):
    #print(img_tiles[index][-12:])
    #print(i[-12:])
    hdf_ds = gdal.Open(img, gdal.GA_ReadOnly)
    array_img=hdf_ds.ReadAsArray()
    check=np.all(array_img==0)
    # if np_where_img all value equal nodata value
    # remove the img
    if check:
        count+=1
        if img[-11:]!=img_tiles[index][-11:]:
            print(index)
            print(img)
            print(img_tiles[index])
            print('---')
        os.remove(img)
        os.remove(img_tiles[index])
        continue
count


In [ ]:
img_tiles_after=glob.glob('../data/GlobalMountainsK3Binary/tilesV2/*.tif')
img_tiles_after=sorted(img_tiles_after)

In [ ]:
len(img_tiles_after)

In [ ]:
len(shp_tiles_after)

In [ ]:
shp_tiles_after=glob.glob('../data/world_shp/tilesV2/*.tif')
shp_tiles_after=sorted(shp_tiles_after)

In [ ]:
counter=0
for index,img in enumerate(shp_tiles_after,0):
    if img[-12:]!=img_tiles_after[index][-12:]:
        print(img)
        print(img_tiles_after[index])

In [ ]:
shp_tiles_after[0][34:]

In [ ]:
for index,img in enumerate(shp_tiles_after,0):
    shp_img = gdal.Open(img, gdal.GA_ReadOnly)
    shp_array=shp_img.ReadAsArray()
    target_img = gdal.Open(img_tiles_after[index], gdal.GA_ReadOnly)
    target_array=target_img.ReadAsArray()
    sum_img=target_array+shp_array
    np_where_img = np.where((sum_img==0),-32768,(sum_img))
    np_where_img = np.where((np_where_img==1),0,(np_where_img))
    np_where_img = np.where((np_where_img==2),1,(np_where_img))
    s_srs = target_img.GetProjectionRef()
    osng = osr.SpatialReference ()
    osng.SetFromUserInput ( s_srs )
    geo_t = target_img.GetGeoTransform ()
    x_size = target_img.RasterXSize # Raster xsize
    y_size = target_img.RasterYSize # Raster ysize
    #create gdal memory file
    mem_drv = gdal.GetDriverByName( 'MEM' )
    dest = mem_drv.Create('', x_size,y_size, 1, gdal.GDT_Int16)
    dest.GetRasterBand(1).WriteArray(np_where_img)
    dest.SetGeoTransform( geo_t )
    dest.SetProjection ( osng.ExportToWkt() )
    dest.GetRasterBand(1).SetNoDataValue(-32768)
    out_name=str(img[34:])
    
    output_path='../data/GlobalMountainsK3Binary/tiles_new_calcV2/k3binary_'+out_name
    #print(output_path)
    gdal.Warp(output_path, dest, format = 'GTiff',srcNodata=-32768,dstNodata=-32768)
    dest=None

In [ ]:
a=glob.glob('../data/GlobalMountainsK3Binary/tiles_new_calcV2/*.tif')
len(a)


In [ ]:
imgplot = plt.imshow(np_where_img)

In [ ]:
img

In [ ]:
def pixelOffset2coord(geotransform,xOffset,yOffset):
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    coordX = originX+pixelWidth*xOffset
    coordY = originY+pixelHeight*yOffset
    return coordX, coordY
